In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

/var/folders/9f/d0blkdy107j9w989n_kd85l40000gn/T/ipykernel_75427/3388212171.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('Heart_Disease_Dataset.csv')
df = pd.DataFrame(data)
df.head()

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0


In [3]:
from sklearn.model_selection import train_test_split
y = df['target']
X = df.drop('target', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_test.head()


,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope
1143,67,0,4,106,223,0,0,142,0,0.3,1
581,51,1,4,131,152,1,2,130,1,1.0,2
1153,52,1,4,128,204,1,0,156,1,1.0,2
388,53,1,4,130,0,0,2,135,1,1.0,2
691,67,0,4,106,223,0,0,142,0,0.3,1


In [4]:
from sklearn.preprocessing import StandardScaler
numerical_cols = ['age', 'resting bp s', 'cholesterol', 'max heart rate', 'oldpeak'] 
scalar = StandardScaler()
X_train[numerical_cols] = scalar.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scalar.transform(X_test[numerical_cols])
X_train.head()


,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope
1028,0.568176,1,1,2.011925,0.764767,0,2,0.742755,0,-0.664514,2
443,-0.280776,1,4,-0.251903,-2.107259,1,1,-0.632886,1,0.233653,2
715,1.098771,0,3,0.394905,1.014075,0,0,-0.309206,0,-0.664514,1
135,-0.493014,1,3,-0.952612,0.535404,0,0,1.390115,0,-0.844148,2
604,0.462057,1,3,0.933912,0.076678,0,1,-0.916106,1,-0.844148,2


In [5]:
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

print(list(X_test.columns))

['age', 'sex', 'chest pain type', 'resting bp s', 'cholesterol', 'fasting blood sugar', 'resting ecg', 'max heart rate', 'exercise angina', 'oldpeak', 'ST slope']


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
logreg_model = LogisticRegression(class_weight='balanced')

logreg_model.fit(X_train_np, y_train_np)
logreg_prediction = logreg_model.predict(X_test_np)

print(confusion_matrix(y_test_np, logreg_prediction))
print(classification_report(y_test_np, logreg_prediction))

[[137  28]
 [ 35 157]]
              precision    recall  f1-score   support

           0       0.80      0.83      0.81       165
           1       0.85      0.82      0.83       192

    accuracy                           0.82       357
   macro avg       0.82      0.82      0.82       357
weighted avg       0.82      0.82      0.82       357



In [7]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(X_train_np, y_train_np)
knn_prediction = knn_model.predict(X_test_np)

print(confusion_matrix(y_test_np, knn_prediction))
print(classification_report(y_test_np, knn_prediction))

[[148  17]
 [ 24 168]]
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       165
           1       0.91      0.88      0.89       192

    accuracy                           0.89       357
   macro avg       0.88      0.89      0.88       357
weighted avg       0.89      0.89      0.89       357



In [8]:
# some hyperparameter tuning for KNN
from sklearn.model_selection import GridSearchCV
import math

parameter_grid = {
    'n_neighbors' : range(int(math.sqrt(len(X_train_np)) - 5), int(math.sqrt(len(X_train_np)) + 5)),
    'weights' : ['uniform', 'distance'],
    'p' : [1, 2]
}

grid_search = GridSearchCV(knn_model, parameter_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_np, y_train_np)

# best n is 1 : figured that out anyways using the classification report but this confirms


GridSearchCV(cv=5, estimator=KNeighborsClassifier(n_neighbors=1),
             param_grid={'n_neighbors': range(23, 33), 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [9]:
print("Best parameters: ", grid_search.best_params_)
best_model = grid_search.best_estimator_


Best parameters:  {'n_neighbors': 24, 'p': 1, 'weights': 'distance'}


In [10]:
improved_knn = best_model
improved_knn.fit(X_train_np, y_train_np )
improved_knn_prediction = improved_knn.predict(X_test_np)

print(confusion_matrix(y_test_np, improved_knn_prediction))
print(classification_report(y_test_np, improved_knn_prediction))



[[155  10]
 [ 16 176]]
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       165
           1       0.95      0.92      0.93       192

    accuracy                           0.93       357
   macro avg       0.93      0.93      0.93       357
weighted avg       0.93      0.93      0.93       357



In [11]:
from sklearn.svm import SVC

hard_svm =SVC(kernel="linear", C=1000)
hard_svm.fit(X_train_np, y_train_np)
hard_svm_prediction = hard_svm.predict(X_test_np)

print(confusion_matrix(y_test_np, hard_svm_prediction))
print(classification_report(y_test_np, hard_svm_prediction))

# SVM should be perfect for this problem given it is a binary classification problem 

[[135  30]
 [ 29 163]]
              precision    recall  f1-score   support

           0       0.82      0.82      0.82       165
           1       0.84      0.85      0.85       192

    accuracy                           0.83       357
   macro avg       0.83      0.83      0.83       357
weighted avg       0.83      0.83      0.83       357



In [12]:
from sklearn.svm import SVC

soft_svm = SVC(kernel="linear", C=0.01)
soft_svm.fit(X_train_np, y_train_np)

soft_svm_prediction = soft_svm.predict(X_test_np)

print(confusion_matrix(y_test_np, soft_svm_prediction))
print(classification_report(y_test_np, soft_svm_prediction))


[[136  29]
 [ 31 161]]
              precision    recall  f1-score   support

           0       0.81      0.82      0.82       165
           1       0.85      0.84      0.84       192

    accuracy                           0.83       357
   macro avg       0.83      0.83      0.83       357
weighted avg       0.83      0.83      0.83       357



In [13]:
from sklearn.tree import DecisionTreeClassifier
# use gini entropy for loss
d_tree = DecisionTreeClassifier(criterion="gini")
d_tree.fit(X_train_np, y_train_np)

d_tree_prediction = d_tree.predict(X_test_np)

print(confusion_matrix(y_test_np, d_tree_prediction))
print(classification_report(y_test_np, d_tree_prediction))

[[147  18]
 [ 40 152]]
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       165
           1       0.89      0.79      0.84       192

    accuracy                           0.84       357
   macro avg       0.84      0.84      0.84       357
weighted avg       0.84      0.84      0.84       357



In [14]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()

rf_model.fit(X_train, y_train)
rf_prediction = rf_model.predict(X_test_np)


print(confusion_matrix(y_test_np, rf_prediction))
print(classification_report(y_test_np, rf_prediction))

[[151  14]
 [ 12 180]]
              precision    recall  f1-score   support

           0       0.93      0.92      0.92       165
           1       0.93      0.94      0.93       192

    accuracy                           0.93       357
   macro avg       0.93      0.93      0.93       357
weighted avg       0.93      0.93      0.93       357



/Users/nikhil/miniconda3/envs/emel/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [15]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_prediction = nb_model.predict(X_test_np)

print(confusion_matrix(y_test_np, nb_prediction))
print(classification_report(y_test_np, nb_prediction))

[[142  23]
 [ 34 158]]
              precision    recall  f1-score   support

           0       0.81      0.86      0.83       165
           1       0.87      0.82      0.85       192

    accuracy                           0.84       357
   macro avg       0.84      0.84      0.84       357
weighted avg       0.84      0.84      0.84       357



/Users/nikhil/miniconda3/envs/emel/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [16]:
from mlxtend.classifier import StackingCVClassifier
stacked_model = StackingCVClassifier(classifiers=[logreg_model, improved_knn, hard_svm, soft_svm, d_tree, rf_model, nb_model],
                                     meta_classifier=LogisticRegression(class_weight='balanced'), cv=3)

stacked_model.fit(X_train_np, y_train_np)

stacked_model_prediction = stacked_model.predict(X_test_np)
print(stacked_model_prediction)

print(confusion_matrix(y_test_np, stacked_model_prediction))
print(classification_report(y_test_np, stacked_model_prediction))

# expected to be perfect

[0 1 1 1 0 1 0 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0
 0 0 1 1 0 1 1 1 0 0 1 1 0 0 0 1 0 1 1 0 0 1 0 0 1 0 0 1 0 1 1 1 1 1 1 0 1
 0 1 0 0 0 0 1 1 0 1 0 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 0
 1 1 0 0 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 1 0 1 0 1 0 1 1 1 0 1
 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 0 1 0 0
 1 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 1 1 0 1 0 0
 0 1 1 1 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 0 1 1 0 0 0 1 1 0
 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 1 1 1 1 1 0 0 1 0
 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1
 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0 1 1 1 0 0 1 0 0 1]
[[156   9]
 [ 16 176]]
              precision    recall  f1-score   support

           0       0.91      0.95      0.93       165
           1       0.95      0.92      0.93       192

    accuracy                           0.93       357
   macro avg       0.93      0.93 

In [17]:
from sklearn.metrics import roc_curve, accuracy_score

false_positive, true_positive, thresholds = roc_curve(y_test_np, stacked_model_prediction)
print("false positive: ", false_positive, "true_positive: ", true_positive, "thresholds: ", thresholds)
print(list(X_test.columns))

print("Accuracy: ", accuracy_score(y_test, stacked_model_prediction)*100, "%")

false positive:  [0.         0.05454545 1.        ] true_positive:  [0.         0.91666667 1.        ] thresholds:  [inf  1.  0.]
['age', 'sex', 'chest pain type', 'resting bp s', 'cholesterol', 'fasting blood sugar', 'resting ecg', 'max heart rate', 'exercise angina', 'oldpeak', 'ST slope']
Accuracy:  92.99719887955182 %


In [18]:
from joblib import dump
dump(stacked_model, "check_my_heart_model.joblib")


['check_my_heart_model.joblib']